In [ ]:
import json
import string
import random
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf

nltk.download("punkt")
nltk.download("wordnet")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data = {"intents": [
      {
        "tag": "greeting",
        "patterns": ["Hallo", "Wie geht's?", "Yo", "Grüß dich", "Alles klar?"],
        "responses": ["Hallo!", "Grüß dich!", "Wie geht's?", "Freut mich, dich zu sehen!", "Hallo, wie läuft's?"]
      },
      {
        "tag": "treatments",
        "patterns": ["Welche ästhetischen Behandlungen bieten Sie an?", "Können Sie mir Informationen über die ästhetischen Behandlungen geben?", "Welche Behandlungen haben Sie?", "Welche Schönheitsbehandlungen sind verfügbar?", "Was für ästhetische Behandlungen gibt es?"],
        "responses": ["Wir bieten eine Vielzahl von ästhetischen Behandlungen an, darunter Lifting, Rhinoplastik und Botox.", "Unsere verfügbaren Behandlungen umfassen Lifting, Rhinoplastik, Botox und viele mehr.", "Wir haben verschiedene ästhetische Behandlungen wie Lifting, Rhinoplastik und Botox im Angebot."]
      },
      {
        "tag": "pricing",
        "patterns": ["0", "Können Sie mir die Preise für die Behandlungen nennen?", "Was kosten die Schönheitsbehandlungen?", "Wie hoch sind die Preise für ästhetische Behandlungen?", "Wie teuer sind die Behandlungen?"],
        "responses": ["Die Preise für unsere ästhetischen Behandlungen variieren je nach Art der Behandlung. Für genaue Preise kontaktieren Sie uns bitte.", "Unsere Behandlungen haben unterschiedliche Preise. Bitte besuchen Sie unsere Website oder kontaktieren Sie uns für detaillierte Informationen.", "Die Kosten für ästhetische Behandlungen hängen von der spezifischen Behandlung ab. Weitere Informationen finden Sie auf unserer Website oder indem Sie uns kontaktieren."]
      },
      {
        "tag": "general_questions",
        "patterns": ["Was ist MedJourney?", "Können Sie mir Informationen über MedJourney geben?", "Was macht Ihre Agentur?", "Was bietet MedJourney an?", "Was genau ist MedJourney?"],
        "responses": ["MedJourney ist eine Plattform für Medizintourismus, die sich auf ästhetische Behandlungen in Sousse, Tunesien, spezialisiert hat.", "Unsere Agentur bietet umfassende Unterstützung für Kunden, die nach ästhetischen Behandlungen in Sousse suchen, einschließlich Beratung und Terminvereinbarung.", "MedJourney hilft deutschen Kunden, ästhetische Behandlungen in Sousse, Tunesien, zu finden und zu buchen."]
      },
      {
        "tag": "goodbye",
        "patterns": ["Tschüss", "Auf Wiedersehen", "Bis bald", "Ciao", "Bis später"],
        "responses": ["Es war schön, mit dir zu reden", "Bis später!", "Wir sprechen uns bald wieder!", "Mach's gut", "Auf Wiedersehen!"]
      }
    ]
  }

In [ ]:
# initialisation de lemmatizer pour obtenir la racine des mots
lemmatizer = WordNetLemmatizer()
# création des listes
words = []
classes = []
doc_X = []
doc_y = []
# parcourir avec une boucle For toutes les intentions
# tokéniser chaque pattern et ajouter les tokens à la liste words, les patterns et
# le tag associé à l'intention sont ajoutés aux listes correspondantes
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])

    # ajouter le tag aux classes s'il n'est pas déjà là
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatiser tous les mots du vocabulaire et les convertir en minuscule
# si les mots n'apparaissent pas dans la ponctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# trier le vocabulaire et les classes par ordre alphabétique et prendre le
# set pour s'assurer qu'il n'y a pas de doublons
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
# liste pour les données d'entraînement
training = []
out_empty = [0] * len(classes)
# création du modèle d'ensemble de mots
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # marque l'index de la classe à laquelle le pattern atguel est associé à
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # ajoute le one hot encoded BoW et les classes associées à la liste training
    training.append([bow, output_row])
# mélanger les données et les convertir en array
random.shuffle(training)
training = np.array(training, dtype=object)
# séparer les features et les labels target
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [ ]:
# définition de quelques paramètres
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200

In [ ]:
# modéle Deep Learning
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])

In [ ]:
# entraînement du modèle
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 1.6734 - accuracy: 0.1600
Epoch 2/200
1/1 [==============================] - 0s 15ms/step - loss: 1.4385 - accuracy: 0.3600
Epoch 3/200
1/1 [==============================] - 0s 11ms/step - loss: 1.3756 - accuracy: 0.4400
Epoch 4/200
1/1 [==============================] - 0s 12ms/step - loss: 1.1809 - accuracy: 0.7200
Epoch 5/200
1/1 [==============================] - 0s 18ms/step - loss: 1.0657 - accuracy: 0.7200
Epoch 6/200
1/1 [==============================] - 0s 11ms/step - loss: 0.9940 - accuracy: 0.7200
Epoch 7/200
1/1 [==============================] - 0s 11ms/step - loss: 0.9684 - accuracy: 0.6000
Epoch 8/200
1/1 [==============================] - 0s 15ms/step - loss: 0.6978 - accuracy: 0.9600
Epoch 9/200
1/1 [==============================] - 0s 11ms/step - loss: 0.6608 - accuracy: 0.9200
Epoch 10/200
1/1 [==============================] - 0s 11ms/step - loss: 0.6087 - accuracy: 0.9200
Epoch 11/200
1/1 [===

In [ ]:
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens
def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)
def pred_class(text, vocab, labels):
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list
def get_response(intents_list, intents_json):
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents:
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
# lancement du chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

KeyboardInterrupt: Interrupted by user

In [ ]:
model.save('my_model.keras')



In [ ]:
import tensorflow as tf

# Load the original model
model = tf.keras.models.load_model('my_model.keras')

# Save the architecture
model_json = model.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)

# Save the weights
model.save_weights("model_weights.h5")


In [ ]:
import tensorflow as tf
print(tf.__version__)


2.15.0
